In [1]:
import numpy as np
from collections import Counter

#### Implementing simple levenstein distance calculation using memory dict to prevent more calls

In [2]:
def call_counter(func):
    def helper(*args, **kwargs):
        helper.calls += 1
        return func(*args, **kwargs)
    helper.calls = 0
    helper.__name__= func.__name__
    return helper

memory = {}

@call_counter
def levenshtein(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    cost = 0 if s[-1] == t[-1] else 1
       
    i1 = (s[:-1], t)
    if not i1 in memory:
        memory[i1] = levenshtein(*i1)
    i2 = (s, t[:-1])
    if not i2 in memory:
        memory[i2] = levenshtein(*i2)
    i3 = (s[:-1], t[:-1])
    if not i3 in memory:
        memory[i3] = levenshtein(*i3)
    res = min([memory[i1]+1, memory[i2]+1, memory[i3]+cost])
    
    return res

In [3]:
print(levenshtein("Python", "Pethno"))
print("The function was called " + str(levenshtein.calls) + " times!")

3
The function was called 49 times!


#### Simple implementation of DBScan clustering to form clusters based on Ld 

In [4]:
words = ['cow','now','bow','apple','sos','combs','andrew','syther','instan']

##### Helper functions

In [5]:
def return_list_ld(list_words,tword,ld):
    list_ids = []
    for word in list_words:
        if(levenshtein(word,tword)<ld):
            list_ids.append(list_words.index(word))
    return list_ids        

In [6]:
visited = np.zeros(words.__len__())

##### clustering forming function..to be optimized later on

In [7]:
def grow(words,ld,cur_cluster,cur_index,min_pts,nearby_points,clusters):
    clusters[cur_index] = cur_cluster
    i=0
    while(i<nearby_points.__len__()):
        point = nearby_points[i]
        if(clusters[point] == -1):
            clusters[point] = cur_cluster
        elif(clusters[point] == 0):
            clusters[point] = cur_cluster
            if(nearby_points<return_list_ld(words,words[cur_index],ld)):
                nearby_points = nearby_points + return_list_ld(words,words[cur_index],ld)
        i +=1
    return clusters

#### Form clusters based on DBSCAN algo (Density-based spatial clustering of applications with noise)

In [13]:
def form_clusters(words,ld,min_pts):
    clusters = np.zeros(words.__len__())
    cur_cluster = 0
    for index in range(0,words.__len__()):
        if(clusters[index]!=0):
            continue
        nearby_points = return_list_ld(words,words[index],ld)
        if(nearby_points.__len__()<min_pts):
            clusters[index]=-1
        else:
            cur_cluster+=1
            clusters = grow(words,ld,cur_cluster,index,min_pts,nearby_points,clusters)
    return clusters       

In [14]:
clusters = form_clusters(words,3,1)

In [15]:
clusters

array([1., 1., 1., 2., 1., 3., 4., 5., 6.])

In [17]:
clusters = list(map(int, clusters))

In [20]:
cluster_dict = {}
for ids in set(clusters):
    cluster_dict["Cluster_{}".format(ids)] = [words[i] for i, x in enumerate(clusters) if x == ids] 

In [21]:
cluster_dict

{'Cluster_1': ['cow', 'now', 'bow', 'sos'],
 'Cluster_2': ['apple'],
 'Cluster_3': ['combs'],
 'Cluster_4': ['andrew'],
 'Cluster_5': ['syther'],
 'Cluster_6': ['instan']}

In [22]:
cluster_dict.__len__()

6

#### Appointing leaders in the clusters based on minimum average distance of one point from the clusters points

In [39]:
def appoint_leaders(cluster_list):
    minld = 100000
    index = 0
    for tword in cluster_list:
        avgld = 0
        for word in cluster_list:
            avgld += levenshtein(tword,word)    
        if(avgld < minld):
            minld = avgld
            index = cluster_list.index(tword)
    return index

In [40]:
cleader_dict = {}
for i in range(1,cluster_dict.__len__()):
    print(cluster_dict["Cluster_{}".format(i)])
    cleader_dict["Cluster_leader_{}".format(i)] = cluster_dict["Cluster_{}".format(i)][appoint_leaders(cluster_dict["Cluster_{}".format(i)])]

['cow', 'now', 'bow', 'sos']
['apple']
['combs']
['andrew']
['syther']


In [41]:
cleader_dict

{'Cluster_leader_1': 'cow',
 'Cluster_leader_2': 'apple',
 'Cluster_leader_3': 'combs',
 'Cluster_leader_4': 'andrew',
 'Cluster_leader_5': 'syther'}

#### Make clusters searchable and device strategy to graph search